## 6-Stock Indian Equity Universe
1. RELIANCE
2. HDFCBANK
3. INFY
4. TAMO
5. BHARTIARTL
6. HUL

In [1]:
import yfinance as yf
import pandas as pd
import os

### 1. Market Data (OHLCV): Fetch daily adjusted prices and volume for the period Jan 1, 2020, to Dec 31, 2025
- Keep Oct 2025 -Dec 2025 data separate for final forward testing.
- Source: Yahoo Finance

In [2]:
stocks = ['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'TATAMOTORS.BO', 'BHARTIARTL.NS', 'HINDUNILVR.NS']

start_date = '2020-01-01'
end_date = '2025-12-31'

# Download data
print("Downloading data...")
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker', auto_adjust=True, threads=False, progress=False)

output_dir = '../data/scrapped'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'market_data_raw.csv')
data.to_csv(output_file)
print(f"Data saved to {output_file}")

train_end_date = '2025-09-30'
test_start_date = '2025-10-01'

train_data = data.loc[:train_end_date]
test_data = data.loc[test_start_date:]

print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TATAMOTORS.BO"}}}
$TATAMOTORS.BO: possibly delisted; no timezone found

1 Failed download:
['TATAMOTORS.BO']: possibly delisted; no timezone found


Data saved to ../data/scrapped\market_data_raw.csv
Train data shape: (1425, 31)
Test data shape: (61, 31)


In [3]:
data.describe()

Ticker  RELIANCE.NS                                                       \
Price          Open         High          Low        Close        Volume   
count   1486.000000  1486.000000  1486.000000  1486.000000  1.486000e+03   
mean    1147.842010  1158.962974  1136.265098  1147.187894  1.812902e+07   
std      240.088986   240.314000   240.114532   240.345686  1.516583e+07   
min      409.864862   424.985412   391.724705   395.482483  0.000000e+00   
25%      997.775472  1009.926259   985.453465   994.736237  9.584944e+06   
50%     1161.403986  1169.942527  1150.964882  1161.183533  1.332284e+07   
75%     1318.708240  1323.090739  1300.431217  1316.421875  2.037365e+07   
max     1592.662006  1596.980044  1573.851280  1589.138062  1.426834e+08   

Ticker HINDUNILVR.NS                                                       \
Price           Open         High          Low        Close        Volume   
count    1486.000000  1486.000000  1486.000000  1486.000000  1.486000e+03   
mean     2284.113400  2304.927895  2260.855240  2282.385853  2.065980e+06   
std       225.620662   225.525352   226.006595   225.547313  4.991731e+06   
min      1671.721915  1715.999794  1587.954988  1661.149292  0.000000e+00   
25%      2137.346461  2156.538630  2115.434914  2134.764526  1.189353e+06   
50%      2309.222175  2330.714966  2290.050993  2307.547485  1.597431e+06   
75%      2446.262236  2465.973045  2424.930921  2445.616150  2.227237e+06   
max      2931.155855  2945.714548  2882.578181  2939.454346  1.856699e+08   

Ticker  ... BHARTIARTL.NS                                          \
Price   ...          High          Low        Close        Volume   
count   ...   1486.000000  1486.000000  1486.000000  1.486000e+03   
mean    ...    990.885124   969.455855   980.128532  1.041675e+07   
std     ...    507.487506   500.426617   504.046741  1.298573e+07   
min     ...    388.200195   345.802415   381.532715  0.000000e+00   
25%     ...    568.267833   550.036744   557.717041  4.394466e+06   
50%     ...    773.312764   754.881312   764.158386  6.592912e+06   
75%     ...   1427.827500  1402.333876  1415.660065  1.186634e+07   
max     ...   2174.500000  2149.899902  2162.699951  1.985470e+08   

Ticker TATAMOTORS.BO                                   
Price           Open High  Low Close Adj Close Volume  
count            0.0  0.0  0.0   0.0       0.0    0.0  
mean             NaN  NaN  NaN   NaN       NaN    NaN  
std              NaN  NaN  NaN   NaN       NaN    NaN  
min              NaN  NaN  NaN   NaN       NaN    NaN  
25%              NaN  NaN  NaN   NaN       NaN    NaN  
50%              NaN  NaN  NaN   NaN       NaN    NaN  
75%              NaN  NaN  NaN   NaN       NaN    NaN  
max              NaN  NaN  NaN   NaN       NaN    NaN  

[8 rows x 31 columns]

### 2. Fundamental Data: Extract quarterly metrics (P/E, Debt/Equity, ROE, EPS), etc. 
- Use a suitable method to align quarterly data with daily market timestamps.
- Source: MoneyControl


In [4]:
import nsepython
import yfinance as yf
import pandas as pd
import numpy as np
import os
import time

# Metrics required:
# Financial Health: EPS, P/E Ratio, Debt-to-Equity, Revenue Growth, ROE, Return on Assets, Net Profit Margin, Operating Margin, PEG Ratio, Price-to-Book (P/B), EV/EBITDA, Earnings Growth
# CAGR, Interest Coverage Ratio, Debt-to-Assets, Current Ratio, Quick Ratio, FCF, OCF, FCF Yield, Asset Turnover, Inventory Turnover, Receivables Turnover.

STOCKS = ['RELIANCE', 'HDFCBANK', 'INFY', 'TATAMOTORS', 'BHARTIARTL', 'HINDUNILVR']
YF_TICKERS = {
    'RELIANCE': 'RELIANCE.NS',
    'HDFCBANK': 'HDFCBANK.NS',
    'INFY': 'INFY.NS',
    'TATAMOTORS': 'TATAMOTORS.BO', # Use BSE as fallback for TATA due to Yahoo delisting error on .NS
    'BHARTIARTL': 'BHARTIARTL.NS',
    'HINDUNILVR': 'HINDUNILVR.NS'
}

OUTPUT_DIR = '../data/scrapped'
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_nse_data(symbol):
    # Map deprecated/changed symbols
    nse_symbol_map = {
        'TATAMOTORS': 'TMPV' # Tata Motors demerged/renamed to TMPV (PV) and TMLCV (CV)
    }
    search_symbol = nse_symbol_map.get(symbol, symbol)
    
    print(f"Fetching NSE data for {search_symbol}...")
    max_retries = 3
    for attempt in range(max_retries):
        try:
            # nse_past_results returns a list of dictionaries
            res = nsepython.nse_past_results(search_symbol)
            # Check if res is a list (success) or dict (failure/empty)
            if isinstance(res, list) and len(res) > 0:
                break
            else:
                print(f"Attempt {attempt+1} failed/empty for {search_symbol}. Result type: {type(res)}")
                if attempt < max_retries - 1:
                    time.sleep(2 * (attempt + 1))
        except Exception as e:
            print(f"Attempt {attempt+1} error for {search_symbol}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 * (attempt + 1))
                
    try:
        if not isinstance(res, list):
             print(f"Final failure for {search_symbol}, using empty DF.")
             return pd.DataFrame()

        df = pd.DataFrame(res)
        if df.empty:
            print(f"No NSE past results for {symbol}")
            return pd.DataFrame()
        
        # Clean date
        if 're_to_dt' in df.columns:
            df['Date'] = pd.to_datetime(df['re_to_dt'], format='%d-%b-%Y', errors='coerce')
        elif 'date' in df.columns:
            df['Date'] = pd.to_datetime(df['date'], errors='coerce')
        
        # Sort by date
        df = df.sort_values('Date')
        return df
    except Exception as e:
        print(f"Error fetching NSE data for {symbol}: {e}")
        return pd.DataFrame()

def get_yahoo_data(ticker_symbol):
    print(f"Fetching Yahoo data for {ticker_symbol}...")
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        # Quarterly Financials (Income Statement)
        fin = ticker.quarterly_financials.T
        if fin.empty: print(f"Warning: Empty Financials for {ticker_symbol}")
            
        # Quarterly Balance Sheet
        bs = ticker.quarterly_balance_sheet.T
        if bs.empty: print(f"Warning: Empty Balance Sheet for {ticker_symbol}")
            
        # Quarterly Cashflow
        cf = ticker.quarterly_cashflow.T
        if cf.empty: print(f"Warning: Empty Cashflow for {ticker_symbol}")

        # Combine
        # Using outer join to keep all dates. Yahoo dates are usually quarter end dates.
        combined = fin.join(bs, how='outer', lsuffix='_fin', rsuffix='_bs')
        combined = combined.join(cf, how='outer', rsuffix='_cf')
        
        # Convert index to datetime
        combined.index = pd.to_datetime(combined.index).tz_localize(None)
        
        # Get historical price data for valuation ratios (P/E, P/B)
        # We need price at the time of the quarter end to calculate historical P/E
        hist = ticker.history(period="5y")
        hist.index = pd.to_datetime(hist.index).tz_localize(None)
        
        return combined, hist, ticker.info
    except Exception as e:
        print(f"Error fetching Yahoo data for {ticker_symbol}: {e}")
        return pd.DataFrame(), pd.DataFrame(), {}

def calculate_metrics(stock, nse_df, yf_combined, yf_hist, yf_info):
    metrics_list = []
    
    # 1. Gather all unique dates
    yf_dates = set(yf_combined.index.tolist()) if not yf_combined.empty else set()
    nse_dates = set(nse_df['Date'].tolist()) if not nse_df.empty else set() # Assuming 'Date' column exists from get_nse_data
    
    all_dates = sorted(list(yf_dates.union(nse_dates)))
    
    for date in all_dates:
        row_data = {'Ticker': stock, 'Date': date}
        
        # --- Get YF Row ---
        yf_row = pd.Series()
        if not yf_combined.empty:
            # Find exact or nearest date within small tolerance?
            # For now, exact match or very close match if we had time differences. 
            # Given we cleaned dates, exact match should work for YF index.
            # But NSE dates might differ by a few days from YF dates (quarter end variants).
            # Let's try exact match first for YF.
             if date in yf_combined.index:
                yf_row = yf_combined.loc[date]
        
        # --- Get NSE Row ---
        nse_row = pd.Series()
        if not nse_df.empty:
            # Check for exact date match in NSE df
            matches = nse_df[nse_df['Date'] == date]
            if not matches.empty:
                nse_row = matches.iloc[0]
            else:
                # Optional: fuzzy match? 
                pass

        # If both are empty, skip (shouldn't happen as we iterate union)
        if yf_row.empty and nse_row.empty:
            continue
            
        # --- Helper to get value safely ---
        def get_val(series, keys, default=np.nan):
            for k in keys:
                if k in series and not pd.isna(series[k]):
                    try:
                        return float(series[k])
                    except:
                        pass
            return default

        # --- Extract Fundamentals (Prioritize YF, Fallback to NSE) ---
        
        # 1. EPS
        eps = get_val(yf_row, ['Basic EPS', 'Diluted EPS'])
        if pd.isna(eps): eps = get_val(nse_row, ['re_basic_eps_for_cont_dic_opr', 're_basic_eps'])
        
        # 2. Net Profit / Net Income
        net_income = get_val(yf_row, ['Net Income', 'Net Income Common Stockholders'])
        if pd.isna(net_income): net_income = get_val(nse_row, ['re_net_profit', 're_proloss_ord_act']) # NSE is in Lakhs?? No, looks like raw numbers or lakhs. Need to check scale. 
        # From previous output: 're_net_profit': '761100' for RELIANCE. Reliance profit is in Crores? 
        # Reliance Q3 net profit ~17000 Cr. 761100 Lakhs = 7611 Cr? 
        # We might need to scale NSE data. Yahoo data is usually in currency (e.g. INR). 
        # Let's assume NSE is in Lakhs and convert to same unit if possible. 
        # But wait, 're_net_profit': '1128300' ~ 11283 Cr. 
        # Yahoo 'Net Income' for Reliance ~ 160-190 billion? No, Yahoo is often in actual currency.
        # Let's just store as is for now and let user handle scaling if mixed.
        # Ideally we should normalize. Yahoo is usually exact value. NSE seems to be Lakhs.
        # Let's multiply NSE by 100,000 to match Yahoo's likely unit?
        # WAIT. 're_net_profit' for Reliance was '1128300'. 11,283 Crores is plausible for quarterly profit.
        # So likely it is in Lakhs. 
        if not pd.isna(net_income) and yf_row.empty and not nse_row.empty:
             net_income = net_income * 100000 


        # 3. Revenue
        revenue = get_val(yf_row, ['Total Revenue', 'Operating Revenue'])
        if pd.isna(revenue): 
            rev_lakhs = get_val(nse_row, ['re_total_inc', 're_net_sale'])
            if not pd.isna(rev_lakhs): revenue = rev_lakhs * 100000
        
        # 4. Total Debt
        total_debt = get_val(yf_row, ['Total Debt', 'Long Term Debt And Capital Lease Obligation']) 
        
        # 5. Total Equity
        equity = get_val(yf_row, ['Stockholders Equity', 'Total Equity Gross Minority Interest'])
        
        # 6. Total Assets
        assets = get_val(yf_row, ['Total Assets'])
        
        # 7. Operating Income / EBIT
        ebit = get_val(yf_row, ['EBIT', 'Operating Income'])
        
        # 8. EBITDA 
        dep_amort = get_val(yf_row, ['Depreciation And Amortization', 'Reconciled Depreciation'])
        if pd.isna(dep_amort): 
            dep_lakhs = get_val(nse_row, ['re_depr_und_exp'])
            if not pd.isna(dep_lakhs): dep_amort = dep_lakhs * 100000
        else:
             pass # default 0 later

        if pd.isna(dep_amort): dep_amort = 0
        
        ebitda = (ebit if not pd.isna(ebit) else 0) + dep_amort
        if ebitda == 0: ebitda = get_val(yf_row, ['EBITDA', 'Normalized EBITDA'])

        # 9. Cash Flow
        fcf = get_val(yf_row, ['Free Cash Flow'])
        ocf = get_val(yf_row, ['Operating Cash Flow', 'Total Cash From Operating Activities'])
        
        # 10. Inventory, Receivables
        inventory = get_val(yf_row, ['Inventory'])
        receivables = get_val(yf_row, ['Accounts Receivable', 'Receivables'])
        
        # --- Calculate Ratios ---
        
        # Get Price
        try:
            # Find price on nearest trading day to quarter end
            idx = yf_hist.index.get_indexer([date], method='nearest')[0]
            close_price = yf_hist.iloc[idx]['Close']
        except:
            close_price = np.nan
        
        # Fallback price if YF history is empty but we have NSE data date? 
        # We don't have price history from NSE here (we didn't fetch it).
        
        shares_val = get_val(pd.Series(yf_info), ['sharesOutstanding'])
        shares_outstanding = shares_val if not pd.isna(shares_val) else np.nan
        
        market_cap = close_price * shares_outstanding if not pd.isna(close_price) and not pd.isna(shares_outstanding) else np.nan
        
        # P/E Ratio
        if not pd.isna(close_price) and not pd.isna(eps) and eps != 0:
            pe_ratio = close_price / (eps * 4) 
        else:
            pe_ratio = np.nan
            
        # Debt-to-Equity
        debt_to_equity = total_debt / equity if not pd.isna(total_debt) and not pd.isna(equity) and equity != 0 else np.nan
        
        # ROE 
        roe = (net_income / equity) * 4 if not pd.isna(net_income) and not pd.isna(equity) and equity != 0 else np.nan
        
        # Return on Assets
        roa = (net_income / assets) * 4 if not pd.isna(net_income) and not pd.isna(assets) and assets != 0 else np.nan
        
        # Net Profit Margin
        npm = net_income / revenue if not pd.isna(net_income) and not pd.isna(revenue) and revenue != 0 else np.nan
        
        # Operating Margin
        op_margin = ebit / revenue if not pd.isna(ebit) and not pd.isna(revenue) and revenue != 0 else np.nan
        
        # Price to Book
        pb_ratio = close_price / (equity / shares_outstanding) if not pd.isna(close_price) and not pd.isna(equity) and shares_outstanding else np.nan
        
        # EV/EBITDA
        cash = get_val(yf_row, ['Cash And Cash Equivalents', 'Cash Financial'])
        if pd.isna(cash): cash = 0
        ev = market_cap + (total_debt if not pd.isna(total_debt) else 0) - cash
        ev_ebitda = ev / (ebitda * 4) if not pd.isna(ev) and ebitda and ebitda != 0 else np.nan
        
        # Current Ratio
        current_assets = get_val(yf_row, ['Current Assets', 'Total Current Assets'])
        current_liabilities = get_val(yf_row, ['Current Liabilities', 'Total Current Liabilities'])
        current_ratio = current_assets / current_liabilities if not pd.isna(current_assets) and current_liabilities else np.nan
        
        # Asset Turnover 
        asset_turnover = (revenue * 4) / assets if not pd.isna(revenue) and not pd.isna(assets) and assets != 0 else np.nan
        
        # Inventory Turnover
        inv_turnover = (revenue * 4) / inventory if not pd.isna(revenue) and not pd.isna(inventory) and inventory != 0 else np.nan
        
        # FCF Yield
        fcf_yield = (fcf * 4) / market_cap if not pd.isna(fcf) and not pd.isna(market_cap) and market_cap != 0 else np.nan
        
        # Populate Metrics
        row_data.update({
            'EPS': eps,
            'P/E Ratio': pe_ratio,
            'Debt-to-Equity': debt_to_equity,
            'Revenue': revenue,
            'Net Profit': net_income,
            'ROE': roe,
            'ROA': roa,
            'Net Profit Margin': npm,
            'Operating Margin': op_margin,
            'P/B Ratio': pb_ratio,
            'EV/EBITDA': ev_ebitda,
            'Current Ratio': current_ratio,
            'Free Cash Flow': fcf,
            'Operating Cash Flow': ocf,
            'Asset Turnover': asset_turnover,
            'Inventory Turnover': inv_turnover,
            'FCF Yield': fcf_yield
        })
        
        metrics_list.append(row_data)

    return pd.DataFrame(metrics_list)

all_metrics = []

for stock in STOCKS:
    print(f"\nProcessing {stock}...")
    
    # Get YF Data
    yf_ticker = YF_TICKERS.get(stock)
    yf_combined, yf_hist, yf_info = get_yahoo_data(yf_ticker)
    
    # Get NSE Data (Fallback/Supplemental)
    nse_df = get_nse_data(stock) 
    
    # Calculate
    stock_metrics = calculate_metrics(stock, nse_df, yf_combined, yf_hist, yf_info)
    
    if not stock_metrics.empty:
        # Calculate Growth Metrics (Requires sorting)
        stock_metrics = stock_metrics.sort_values('Date')
        
        # Revenue Growth
        # Handle pct_change deprecation by filling method or just ignoring warning for now (it's a warning)
        stock_metrics['Revenue Growth'] = stock_metrics['Revenue'].pct_change(fill_method=None)
        
        # Earnings Growth
        stock_metrics['Earnings Growth'] = stock_metrics['EPS'].pct_change(fill_method=None)
        
        all_metrics.append(stock_metrics)

if all_metrics:
    final_df = pd.concat(all_metrics, ignore_index=True)
    
    output_path = os.path.join(OUTPUT_DIR, 'fundamental_data.csv')
    final_df.to_csv(output_path, index=False)
    print(f"\n\nSUCCESS: Saved fundamental data to {output_path}")
    print(final_df.head())
    print(f"Total Rows: {len(final_df)}")
    print(f"Columns: {final_df.columns.tolist()}")
else:
    print("FAILED: No data collected.")



Processing RELIANCE...
Fetching Yahoo data for RELIANCE.NS...
Fetching NSE data for RELIANCE...
Attempt 1 failed/empty for RELIANCE. Result type: <class 'dict'>
Attempt 2 failed/empty for RELIANCE. Result type: <class 'dict'>
Attempt 3 failed/empty for RELIANCE. Result type: <class 'dict'>
Final failure for RELIANCE, using empty DF.

Processing HDFCBANK...
Fetching Yahoo data for HDFCBANK.NS...
Fetching NSE data for HDFCBANK...
Attempt 1 failed/empty for HDFCBANK. Result type: <class 'dict'>
Attempt 2 failed/empty for HDFCBANK. Result type: <class 'dict'>
Attempt 3 failed/empty for HDFCBANK. Result type: <class 'dict'>
Final failure for HDFCBANK, using empty DF.

Processing INFY...
Fetching Yahoo data for INFY.NS...
Fetching NSE data for INFY...
Attempt 1 failed/empty for INFY. Result type: <class 'dict'>
Attempt 2 failed/empty for INFY. Result type: <class 'dict'>
Attempt 3 failed/empty for INFY. Result type: <class 'dict'>
Final failure for INFY, using empty DF.

Processing TATAMOTO

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TATAMOTORS.BO"}}}
$TATAMOTORS.BO: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Fetching NSE data for TMPV...
Attempt 1 failed/empty for TMPV. Result type: <class 'dict'>
Attempt 2 failed/empty for TMPV. Result type: <class 'dict'>
Attempt 3 failed/empty for TMPV. Result type: <class 'dict'>
Final failure for TMPV, using empty DF.

Processing BHARTIARTL...
Fetching Yahoo data for BHARTIARTL.NS...
Fetching NSE data for BHARTIARTL...
Attempt 1 failed/empty for BHARTIARTL. Result type: <class 'dict'>
Attempt 2 failed/empty for BHARTIARTL. Result type: <class 'dict'>
Attempt 3 failed/empty for BHARTIARTL. Result type: <class 'dict'>
Final failure for BHARTIARTL, using empty DF.

Processing HINDUNILVR...
Fetching Yahoo data for HINDUNILVR.NS...
Fetching NSE data for HINDUNILVR...
Attempt 1 failed/empty for HINDUNILVR. Result type: <class 'dict'>
Attempt 2 failed/empty for HINDUNILVR. Result type: <class 'dict'>
Attempt 3 failed/empty for HINDUNILVR. Result type: <class 'dict'>
Final failure for HINDUNILVR, using empty DF.


SUCCESS: Saved fundamental data to ../data/sc

### 3. Macro Indicators: Inflation, Integrate daily/monthly USD-INR rates, India 10Y Bond Yields, and Crude Oil prices, etc.
- Sources: RBI data, Yahoo Finance


### 4. Alternative Data (Sentiment): Scrape or API-fetch financial news headlines. 
- Use a pre-trained transformer model to generate daily sentiment polarity scores.
- Sources: Google News API + FinBERT
